# Zadanie 3 (7 punktów)

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry  Czwórki (ang. Connect Four).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four.

Należy zaimplementować co najmniej dwie heurystyki do ewaluacji planszy.  

Implementację algorytmu Minimax (klasa `MiniMaxSolver`) należy przetestować używając różną głębokość przeszukiwania symulując grę "komputer vs komputer". W eksperymentach należy również zademonstrować różnice pomiędzy heurystykami.  

W ramach zadania można zaimplementować dowolną liczbę dodatkowych metod w klasie `MiniMaxSolver`.

Punktacja:

- Działająca metoda Minimax oraz heurystyki do ewaluacji planszy. - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta. - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania - wykresy. - **2pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów
- Jakość kodu. - **1.5pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [319]:
from typing import Tuple, List, Callable

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove, ConnectFourState

Wielkość planszy

In [320]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [321]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour, heuristic_function: Callable[[ConnectFourState, Player, Player], float]):
        self.game = game
        self.heuristic = heuristic_function

    def get_best_move(self, depth)->int:
        is_maximizing = self.game.first_player.char == self.game.get_current_player().char
        result = self.minimax(depth, float("-inf"), float("inf"), is_maximizing, self.game.state)
        return result[0]

    def is_valid_move(self, col_index:int)->bool:
        pass

    def eval_position(self, position: ConnectFourState) -> float:
        return self.heuristic(position, self.game.first_player, self.game.second_player)

    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool, position: ConnectFourState)-> Tuple[int, float]:
        """Returns column index and score"""
        if depth == 0 or position.is_finished():
            return (None, self.eval_position(position))

        column = 0
        if is_maximizing_player:
            for child in position.get_moves():
                new_position = position.make_move(child)
                eval = self.minimax(depth - 1, alpha, beta, False, new_position)
                if eval[0] is None:
                    eval = (child.column, eval[1])
                if eval[1] > alpha:
                    alpha = eval[1]
                    column = eval[0]
                if alpha >= beta:
                    break
            return (column, alpha)
        else:
            for child in position.get_moves():
                new_position = position.make_move(child)
                eval = self.minimax(depth - 1, alpha, beta, True, new_position)
                if eval[0] is None:
                    eval = (child.column, eval[1])
                if eval[1] < beta:
                    beta = eval[1]
                    column = eval[0]
                if alpha >= beta:
                    break
            return (column, beta)


In [322]:
from typing import Optional

def first_heuristic(state: ConnectFourState, max_player:Player, min_player:Player):

    def score_line(line, player: Player) -> int:
        """Scores a single line based on the player's streaks."""
        line = list(map(lambda x: x.char if x is not None else None, line))
        score = 0
        streak_count = line.count(player.char)
        empty_count = line.count(None)

        if streak_count == 4:  # Winning streak
            score += 10000000
        elif streak_count == 3 and empty_count == 1:  # Potential winning move
            score += 100000
        elif streak_count == 2 and empty_count == 2:  # Building streak
            score += 1000

        return score

    fields = state.fields

    lines = []
    # Vertical
    for column_id in range(COLUMN_COUNT):  # verticals
        for start_row_id in range(ROW_COUNT - 3):
            lines.append([fields[column_id][start_row_id + i] for i in range(4)])

    # Horizontal
    for start_column_id in range(COLUMN_COUNT - 3):  # horizontals
        for row_id in range(ROW_COUNT):
            lines.append([fields[start_column_id + i][row_id] for i in range(4)])


    # Diagonal
    for start_column_id in range(COLUMN_COUNT - 3):  # diagonals
        for start_row_id in range(ROW_COUNT - 3):
            lines.append([fields[start_column_id + i][start_row_id + i] for i in range(4)])
            lines.append([fields[start_column_id - i + 3][start_row_id + i] for i in range(4)])

    result = 0
    for line in lines:
        result += score_line(line, max_player)
        result -= score_line(line, min_player)

    return result

Rozgrywka

In [323]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
minimax = MinMaxSolver(game, first_heuristic)
while(not game.is_finished()):
    print(game)
    game.make_move(ConnectFourMove(minimax.get_best_move(7)))
print(game)



Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][a][ ][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][a][ ][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][a][ ][ ][a][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[b][ ][a][ ][ ][a][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[b][ ][a][ ][a][a][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][

In [324]:
def first_heuristic(state: ConnectFourState, max_player:Player, min_player:Player):

    def score_line(line: List[Optional[Player]], player: Player) -> int:
        """Scores a single line based on the player's streaks."""

        score = 0
        streak_count = line.count(player.char)
        empty_count = line.count(None)

        if streak_count == 4:  # Winning streak
            score += 1000000
        elif streak_count == 3 and empty_count == 1:  # Potential winning move
            score += 10000
        elif streak_count == 2 and empty_count == 2:  # Building streak
            score += 5
        return score

    fields = state.fields

    lines = []
    # Vertical
    for column_id in range(COLUMN_COUNT):  # verticals
        for start_row_id in range(ROW_COUNT - 3):
            lines.append(fields[column_id][start_row_id:(start_row_id + 4)])

    # Horizontal
    for start_column_id in range(COLUMN_COUNT - 3):  # horizontals
        for row_id in range(ROW_COUNT):
            lines.append([fields[start_column_id + i][row_id] for i in range(4)])


    # Diagonal
    for start_column_id in range(COLUMN_COUNT - 3):  # diagonals
        for start_row_id in range(ROW_COUNT - 3):
            lines.append([fields[start_column_id + i][start_row_id + i] for i in range(4)])
            lines.append([fields[start_column_id - i + 3][start_row_id + i] for i in range(4)])

    result = 0
    for line in lines:
        result += score_line(line, max_player)
        result -= score_line(line, min_player)

    return result

pola = [
    [None, None, None, None, None, None],  # Column 0
    [None, None, None, None, '1', None],  # Column 1
    [None, None, None, '2', '1', '1'],    # Column 2
    [None, None, None, '1', '2', '2'],    # Column 3
    [None, None, '2', '1', '2', '1'],  # Column 4
    [None, None, None, None, None, '1'], # Column 5
    [None, None, None, None, None, None], # Column 6
]

max_player = Player('1')
min_player = Player('2')
test = Player('1')
print(test == max_player)
state = ConnectFourState((7, 6), max_player, min_player, pola)
# Evaluate heuristic
score = first_heuristic(state, max_player, min_player)
# print(f"Heuristic Score: {score}")

False
